In [1]:
import pandas as pd
import numpy as np
import math
import sqlite3
import os

In [2]:
#NHL_ALL_SHIFTS_DF = pd.read_csv(r"C:\Users\jorda\Documents\Concordia Boot Camp\Lectures\NHL_ALL_SHIFTS.csv")
#NHL_ALL_SHIFTS_DF

,Unnamed: 0,Game_Id,Period,Team,Player,Player_Id,Start,End,Duration,Date,Season_id
0,0,20001,1,MTL,BRIAN GIONTA,8467407,0.0,34.0,34.0,2013-10-01,2013
1,1,20001,1,MTL,TOMAS PLEKANEC,8469521,0.0,29.0,29.0,2013-10-01,2013
2,2,20001,1,MTL,JOSH GORGES,8470324,0.0,55.0,55.0,2013-10-01,2013
3,3,20001,1,TOR,DION PHANEUF,8470602,0.0,44.0,44.0,2013-10-01,2013
4,4,20001,1,MTL,RENE BOURQUE,8471504,0.0,34.0,34.0,2013-10-01,2013
...,...,...,...,...,...,...,...,...,...,...,...
10000090,921222,21213,3,ANA,JAKOB SILFVERBERG,8475164,1174.0,1200.0,26.0,2023-04-01,2022
10000091,921223,21213,3,EDM,NICK BJUGSTAD,8475760,1174.0,1200.0,26.0,2023-04-01,2022
10000092,921224,21213,3,EDM,MATTIAS JANMARK,8477406,1174.0,1200.0,26.0,2023-04-01,2022
10000093,921225,21213,3,EDM,WARREN FOEGELE,8477998,1174.0,1200.0,26.0,2023-04-01,2022


In [2]:
NHL_SHIFTS_BY_SEASON = pd.read_csv(r"C:\Users\jorda\Documents\Concordia Boot Camp\Lectures\NHL_Player_TOI.csv")
NHL_SHIFTS_BY_SEASON

,Player,Player_Id,Season_id,TOI_seconds,Game_Count
0,A.J. GREER,8478421,2016,4097.0,5
1,A.J. GREER,8478421,2017,7541.0,17
2,A.J. GREER,8478421,2018,5641.0,15
3,A.J. GREER,8478421,2020,513.0,1
4,A.J. GREER,8478421,2021,4242.0,9
...,...,...,...,...,...
10062,ZEMGUS GIRGENSONS,8476878,2018,58801.0,72
10063,ZEMGUS GIRGENSONS,8476878,2019,57106.0,69
10064,ZEMGUS GIRGENSONS,8476878,2021,50180.0,56
10065,ZEMGUS GIRGENSONS,8476878,2022,56521.0,73


In [3]:
Player_Bio_Data = pd.read_csv(r"C:\Users\jorda\Documents\Concordia Boot Camp\Lectures\Player_Bios.csv")
Player_Bio_Data

,Player_ID,Position,Handedness,Birth_Year,Birth_Month,Draft_Year,Draft_Team,Draft_Position
0,8470638,C,Right,1985,July,2003,BOS,45
1,8478402,C,Left,1997,January,2015,EDM,1
2,8477409,C,Left,1995,August,2013,TOR,82
3,8479314,W,Left,1997,December,2016,CGY,6
4,8480835,C,Left,2000,February,2018,CAR,42
...,...,...,...,...,...,...,...,...
2066,8481640,C,Left,1998,October,undrafted,undrafted,undrafted
2067,8480293,D,Left,1997,April,2017,PHI,196
2068,8479991,C,Left,1999,March,2017,VGK,62
2069,8478147,W,Left,1996,August,2015,TBL,180


Index(['Season_id', 'Game_Id', 'Date', 'Period', 'Seconds_Elapsed', 'Event',
       'Ev_Zone', 'Strength', 'p1_name', 'p2_ID', 'p2_name', 'p1_ID',
       'p3_name', 'p3_ID', 'Ev_Team', 'xG', 'Is_Rebound', 'Off_Rush',
       'Off_FO_Win', 'Off_Takeaway', 'DEFLECTED', 'Shot_Distance',
       'Royal_Road', 'BLOCK', 'FAC', 'GIVE', 'GOAL', 'HIT', 'MISS', 'PENL',
       'SHOT', 'TAKE'],
      dtype='object')

In [4]:
Player_Event_DF_Final = pd.read_csv(r"C:\Users\jorda\Documents\Concordia Boot Camp\Lectures\NHL_Player_Metrics_PBP.csv")
Player_Event_DF_Final

,Player,Player_Id,Season_id,Games_Played,DZ_Blocks,DZ_Faceoff_Won,DZ_Giveaways,DZ_Hits,DZ_Penalties_Taken,DZ_Takeaways,...,Actual_to_Expected_Goals_%,Average_Shot_Distance,Total_Giveaways,Total_Hits,Total_Penalties_Taken,Total_Takeaways,DZ_Faceoff_Win_Pct,NZ_Faceoff_Win_Pct,OZ_Faceoff_Win_Pct,Faceoff_Win_Pct
0,CONNOR MCDAVID,8478402,2021,96,29,209,22,35,12,26,...,56,16.58403562,88,126,21,87,0.55291,0.582796,0.645503,0.592957
1,ALEX OVECHKIN,8471214,2017,106,35,1,28,48,9,13,...,63,23.84974153,88,220,18,39,0.25000,0.333333,1.000000,0.375000
2,ALEX OVECHKIN,8471214,2015,91,38,0,13,65,10,4,...,56,23.74382871,53,284,24,30,0.00000,0.000000,0.000000,0.000000
3,ALEX OVECHKIN,8471214,2014,95,39,0,26,84,11,11,...,57,23.65119739,72,316,27,50,0.00000,1.000000,1.000000,0.500000
4,ZACH HYMAN,8475786,2022,74,16,1,4,13,7,7,...,33,14.58374331,30,74,18,32,0.25000,0.333333,0.800000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9963,YANN DANIS,8471186,2015,1,0,0,1,0,0,0,...,0,#DIV/0!,0,0,0,0,0.00000,0.000000,0.000000,0.000000
9964,YAROSLAV ASKAROV,8482137,2022,1,0,0,1,0,0,0,...,0,#DIV/0!,0,0,0,0,0.00000,0.000000,0.000000,0.000000
9965,ZAC DALPE,8474610,2021,1,1,0,0,0,0,0,...,0,#DIV/0!,0,2,0,0,0.00000,0.000000,0.500000,0.250000
9966,ZACH SAWCHENKO,8479313,2021,1,0,0,1,0,0,0,...,0,#DIV/0!,0,0,0,0,0.00000,0.000000,0.000000,0.000000


In [5]:
Player_RAPM_Values = pd.read_csv(r"C:\Users\jorda\Documents\Concordia Boot Camp\Lectures\NHL_RAPM_ALL_SEASON_OVER_SEASON.csv")
Player_RAPM_Values['Offensive_RAPM'] = Player_RAPM_Values['coef_x']
Player_RAPM_Values['Defensive_RAPM'] = Player_RAPM_Values['coef_y']
Player_RAPM_Values = Player_RAPM_Values[['player_id','player_name','Offensive_RAPM','Defensive_RAPM','total','Season']]
Player_RAPM_Values

,player_id,player_name,Offensive_RAPM,Defensive_RAPM,total,Season
0,8470638.0,PATRICE BERGERON,0.469415,-1.049825,1.519240,2021
1,8479314.0,MATTHEW TKACHUK,1.219636,-0.222967,1.442604,2022
2,8478402.0,CONNOR MCDAVID,1.129378,-0.119175,1.248553,2021
3,8475193.0,TOMAS TATAR,0.780979,-0.379501,1.160480,2022
4,8478427.0,SEBASTIAN AHO,1.134566,-0.016683,1.151249,2018
...,...,...,...,...,...,...
9088,8478062.0,DYSIN MAYO,-0.540083,0.392107,-0.932190,2021
9089,8473989.0,LUKE GAZDIC,-0.535565,0.412858,-0.948423,2013
9090,8477384.0,JOSH BROWN,-0.459986,0.543817,-1.003804,2022
9091,8467334.0,MANNY MALHOTRA,-0.506978,0.505335,-1.012312,2014


In [ ]:
#Player_Event_DF = pd.read_csv(r"C:\Users\jorda\Documents\Concordia Boot Camp\Lectures\NHL_EVENTS_WITH_PLAYERS.csv")
#Player_Event_DF['p1_name'] = Player_Event_DF['p1_name'].fillna(0)
#Player_Event_DF['Is_Rebound'] = Player_Event_DF['Is_Rebound'].fillna(0)
#Player_Event_DF['Off_Rush'] = Player_Event_DF['Off_Rush'].fillna(0)
#Player_Event_DF['Off_FO_Win'] = Player_Event_DF['Off_FO_Win'].fillna(0)
#Player_Event_DF['Off_Takeaway'] = Player_Event_DF['Off_Takeaway'].fillna(0)
#Player_Event_DF['DEFLECTED'] = Player_Event_DF['DEFLECTED'].fillna(0)
#Player_Event_DF['Shot_Distance'] = Player_Event_DF['Shot_Distance'].fillna(0)
#Player_Event_DF['Royal_Road'] = Player_Event_DF['Royal_Road'].fillna(0)
#Player_Event_DF = Player_Event_DF[Player_Event_DF['p1_name']!=0]
#Player_Event_DF

In [10]:
#Event_cats = pd.get_dummies(Player_Event_DF['Event'])
#Player_Event_DF = pd.concat([Player_Event_DF.reset_index(drop=True),Event_cats.reset_index(drop=True)], axis=1)

#Player_Event_DF

,Season_id,Game_Id,Date,Period,Seconds_Elapsed,Event,Ev_Zone,Strength,p1_name,p2_ID,...,Royal_Road,BLOCK,FAC,GIVE,GOAL,HIT,MISS,PENL,SHOT,TAKE
0,2013,20001,2013-10-01,1,0.0,FAC,Neu,5x5,TYLER BOZAK,8469521.0,...,0.0,0,1,0,0,0,0,0,0,0
1,2013,20001,2013-10-01,1,74.0,MISS,Off,5x5,CODY FRANSON,NaN,...,0.0,0,0,0,0,0,1,0,0,0
2,2013,20001,2013-10-01,1,96.0,SHOT,Off,5x5,BRENDAN GALLAGHER,NaN,...,0.0,0,0,0,0,0,0,0,1,0
3,2013,20001,2013-10-01,1,100.0,HIT,Off,5x5,ALEX GALCHENYUK,8474581.0,...,0.0,0,0,0,0,1,0,0,0,0
4,2013,20001,2013-10-01,1,105.0,BLOCK,Def,5x5,JARRED TINORDI,8471245.0,...,0.0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3321274,2022,21213,2023-04-01,3,1159.0,GIVE,Def,5x5,BRETT KULAK,NaN,...,0.0,0,0,1,0,0,0,0,0,0
3321275,2022,21213,2023-04-01,3,1174.0,PENL,Def,5x5,MASON MCTAVISH,8480011.0,...,0.0,0,0,0,0,0,0,1,0,0
3321276,2022,21213,2023-04-01,3,1174.0,FAC,Def,5x4,DEREK GRANT,8477406.0,...,0.0,0,1,0,0,0,0,0,0,0
3321277,2022,21213,2023-04-01,3,1179.0,SHOT,Def,5x4,NATHAN BEAULIEU,NaN,...,1.0,0,0,0,0,0,0,0,1,0


In [6]:
NHL_DATA_DB = "../NHL_DATA.sqlite"
NHL_DATA_conn = sqlite3.connect(NHL_DATA_DB)

Player_Event_DF.to_sql(name='Player_Events', if_exists='replace', con=NHL_DATA_conn)
Player_Event_DF_Final.to_sql(name='Player_Metrics', if_exists='replace', con=NHL_DATA_conn)
Player_Bio_Data.to_sql(name='Player_Bio', if_exists='replace', con=NHL_DATA_conn)
Player_RAPM_Values.to_sql(name='Player_RAPM', if_exists='replace', con=NHL_DATA_conn)
NHL_SHIFTS_BY_SEASON.to_sql(name='Player_Shifts', if_exists='replace', con=NHL_DATA_conn)

10067

In [7]:
pd.read_sql_query(
"""
SELECT 
*
FROM Player_Metrics
 """, NHL_DATA_conn)

,index,Player,Player_Id,Season_id,Games_Played,DZ_Blocks,DZ_Faceoff_Won,DZ_Giveaways,DZ_Hits,DZ_Penalties_Taken,...,Actual_to_Expected_Goals_%,Average_Shot_Distance,Total_Giveaways,Total_Hits,Total_Penalties_Taken,Total_Takeaways,DZ_Faceoff_Win_Pct,NZ_Faceoff_Win_Pct,OZ_Faceoff_Win_Pct,Faceoff_Win_Pct
0,0,CONNOR MCDAVID,8478402,2021,96,29,209,22,35,12,...,56,16.58403562,88,126,21,87,0.55291,0.582796,0.645503,0.592957
1,1,ALEX OVECHKIN,8471214,2017,106,35,1,28,48,9,...,63,23.84974153,88,220,18,39,0.25000,0.333333,1.000000,0.375000
2,2,ALEX OVECHKIN,8471214,2015,91,38,0,13,65,10,...,56,23.74382871,53,284,24,30,0.00000,0.000000,0.000000,0.000000
3,3,ALEX OVECHKIN,8471214,2014,95,39,0,26,84,11,...,57,23.65119739,72,316,27,50,0.00000,1.000000,1.000000,0.500000
4,4,ZACH HYMAN,8475786,2022,74,16,1,4,13,7,...,33,14.58374331,30,74,18,32,0.25000,0.333333,0.800000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9963,9963,YANN DANIS,8471186,2015,1,0,0,1,0,0,...,0,#DIV/0!,0,0,0,0,0.00000,0.000000,0.000000,0.000000
9964,9964,YAROSLAV ASKAROV,8482137,2022,1,0,0,1,0,0,...,0,#DIV/0!,0,0,0,0,0.00000,0.000000,0.000000,0.000000
9965,9965,ZAC DALPE,8474610,2021,1,1,0,0,0,0,...,0,#DIV/0!,0,2,0,0,0.00000,0.000000,0.500000,0.250000
9966,9966,ZACH SAWCHENKO,8479313,2021,1,0,0,1,0,0,...,0,#DIV/0!,0,0,0,0,0.00000,0.000000,0.000000,0.000000


In [8]:
pd.read_sql_query(
"""
SELECT 
*
FROM Player_Bio
 """, NHL_DATA_conn)

,index,Player_ID,Position,Handedness,Birth_Year,Birth_Month,Draft_Year,Draft_Team,Draft_Position
0,0,8470638,C,Right,1985,July,2003,BOS,45
1,1,8478402,C,Left,1997,January,2015,EDM,1
2,2,8477409,C,Left,1995,August,2013,TOR,82
3,3,8479314,W,Left,1997,December,2016,CGY,6
4,4,8480835,C,Left,2000,February,2018,CAR,42
...,...,...,...,...,...,...,...,...,...
2066,2066,8481640,C,Left,1998,October,undrafted,undrafted,undrafted
2067,2067,8480293,D,Left,1997,April,2017,PHI,196
2068,2068,8479991,C,Left,1999,March,2017,VGK,62
2069,2069,8478147,W,Left,1996,August,2015,TBL,180


In [9]:
pd.read_sql_query(
"""
SELECT 
*
FROM Player_RAPM
 """, NHL_DATA_conn)

,index,player_id,player_name,Offensive_RAPM,Defensive_RAPM,total,Season
0,0,8470638.0,PATRICE BERGERON,0.469415,-1.049825,1.519240,2021
1,1,8479314.0,MATTHEW TKACHUK,1.219636,-0.222967,1.442604,2022
2,2,8478402.0,CONNOR MCDAVID,1.129378,-0.119175,1.248553,2021
3,3,8475193.0,TOMAS TATAR,0.780979,-0.379501,1.160480,2022
4,4,8478427.0,SEBASTIAN AHO,1.134566,-0.016683,1.151249,2018
...,...,...,...,...,...,...,...
9088,9088,8478062.0,DYSIN MAYO,-0.540083,0.392107,-0.932190,2021
9089,9089,8473989.0,LUKE GAZDIC,-0.535565,0.412858,-0.948423,2013
9090,9090,8477384.0,JOSH BROWN,-0.459986,0.543817,-1.003804,2022
9091,9091,8467334.0,MANNY MALHOTRA,-0.506978,0.505335,-1.012312,2014


In [10]:
pd.read_sql_query(
"""
SELECT 
*
FROM Player_Shifts
 """, NHL_DATA_conn)

,index,Player,Player_Id,Season_id,TOI_seconds,Game_Count
0,0,A.J. GREER,8478421,2016,4097.0,5
1,1,A.J. GREER,8478421,2017,7541.0,17
2,2,A.J. GREER,8478421,2018,5641.0,15
3,3,A.J. GREER,8478421,2020,513.0,1
4,4,A.J. GREER,8478421,2021,4242.0,9
...,...,...,...,...,...,...
10062,10062,ZEMGUS GIRGENSONS,8476878,2018,58801.0,72
10063,10063,ZEMGUS GIRGENSONS,8476878,2019,57106.0,69
10064,10064,ZEMGUS GIRGENSONS,8476878,2021,50180.0,56
10065,10065,ZEMGUS GIRGENSONS,8476878,2022,56521.0,73


In [87]:
#WAS USED TO GET TIME ON ICE FOR EACH PLAYER FOR EACH SEASON. NOT TO BE USED ANYMORE

NHL_SHIFTS_BY_SEASON = pd.read_sql_query(
"""
SELECT distinct

Player
,Player_Id
,Season_id
,sum(Duration) as "TOI_seconds"
,count(distinct Game_Id) as "Game_Count"
FROM Player_Shifts
group by Player, Player_Id, Season_id
 """, NHL_DATA_conn)
NHL_SHIFTS_BY_SEASON

,Player,Player_Id,Season_id,TOI_seconds,Game_Count
0,A.J. GREER,8478421,2016,4097.0,5
1,A.J. GREER,8478421,2017,7541.0,17
2,A.J. GREER,8478421,2018,5641.0,15
3,A.J. GREER,8478421,2020,513.0,1
4,A.J. GREER,8478421,2021,4242.0,9
...,...,...,...,...,...
10062,ZEMGUS GIRGENSONS,8476878,2018,58801.0,72
10063,ZEMGUS GIRGENSONS,8476878,2019,57106.0,69
10064,ZEMGUS GIRGENSONS,8476878,2021,50180.0,56
10065,ZEMGUS GIRGENSONS,8476878,2022,56521.0,73


In [88]:
NHL_SHIFTS_BY_SEASON.to_csv('NHL_Player_TOI.csv', index=False)


In [ ]:
# WAS USED TO MAKE A MORE CLEAN PLAYER METRICS TALBE. NOT TO BE USED ANYMORE.

NHL_PLAYER_STATS = pd.read_sql_query(
"""
with p1_stats as (
SELECT 
p1_name as Player
,p1_ID as Player_Id
,Season_id


,count(distinct Game_Id) as "Games_Played"


,sum(BLOCK) filter(where Ev_Zone='Def') as "DZ_Blocks"
,sum(FAC) filter(where Ev_Zone='Def') as "DZ_Faceoff_Won"
,sum(GIVE) filter(where Ev_Zone='Def') as "DZ_Giveaways"
,sum(HIT) filter(where Ev_Zone='Def') as "DZ_Hits"
,sum(PENL) filter(where Ev_Zone='Def') as "DZ_Penalties_Taken"
,sum(TAKE) filter(where Ev_Zone='Def') as "DZ_Takeaways"


,sum(FAC) filter(where Ev_Zone='Neu') as "NZ_Faceoff_Won"
,sum(GIVE) filter(where Ev_Zone='Neu') as "NZ_Giveaways"
,sum(HIT) filter(where Ev_Zone='Neu') as "NZ_Hits"
,sum(PENL) filter(where Ev_Zone='Neu') as "NZ_Penalties_Taken"
,sum(TAKE) filter(where Ev_Zone='Neu') as "NZ_Takeaways"


,sum(Is_Rebound) as "Rebound_Shot_Attempts"
,sum(Off_Rush) as "Shot_Attempts_OFf_Rush"
,sum(Off_FO_Win) as "Shot_Attempts_Off_FaceOff"
,sum(Off_Takeaway) as "Shot_Attempts_Off_Takeaway"
,sum(DEFLECTED) as "DEFLECTED_Shot_Attempts"
,sum(Royal_Road) as "Royal_Road_Shot_Attempts"
,sum(Shot_Distance) as "Shot_Distance_Total"


,sum(FAC) filter(where Ev_Zone='Off') as "OZ_Faceoff_Won"
,sum(GIVE) filter(where Ev_Zone='Off') as "OZ_Giveaways"
,sum(HIT) filter(where Ev_Zone='Off') as "OZ_Hits"
,sum(PENL) filter(where Ev_Zone='Off') as "OZ_Penalties_Taken"
,sum(TAKE) filter(where Ev_Zone='Off') as "OZ_Takeaways"
,sum(GOAL) filter(where Ev_Zone='Off') as "Goals"
,sum(xG) filter(where Ev_Zone='Off') as "xG"


,sum(SHOT) filter(where Ev_Zone='Off') as "Shots_on_Net (no goal)"
,sum(MISS) filter(where Ev_Zone='Off') as "Shots_for_miss"


FROM Player_Events
GROUP BY p1_name, p1_ID, Season_id
)

,p2_stats as (

SELECT 
p2_name as Player2
,p2_ID as Player_Id2
,Season_id as Season_id2


,sum(FAC) filter(where Ev_Zone='Def') as "DZ_Faceoff_Lost"
,sum(HIT) filter(where Ev_Zone='Def') as "DZ_Hits_Recieved"
,sum(PENL) filter(where Ev_Zone='Def') as "DZ_Penalties_Drawn"


,sum(FAC) filter(where Ev_Zone='Neu') as "NZ_Faceoff_Lost"
,sum(HIT) filter(where Ev_Zone='Neu') as "NZ_Hits_Recieved"
,sum(PENL) filter(where Ev_Zone='Neu') as "NZ_Penalties_Drawn"


,sum(FAC) filter(where Ev_Zone='Off') as "OZ_Faceoff_Lost"
,sum(HIT) filter(where Ev_Zone='Off') as "OZ_Hits_Recieved"
,sum(PENL) filter(where Ev_Zone='Off') as "OZ_Penalties_Drawn"
,sum(GOAL) filter(where Ev_Zone='Off') as "1st_Assists"
,sum(BLOCK) filter(where Ev_Zone='Def') as "Shot_Attempts_Leading_to_Block"


FROM Player_Events
GROUP BY p2_name, p2_ID, Season_id
)

,p3_stats as (
SELECT


p3_name as Player3
,p3_ID as Player_Id3
,Season_id as Season_id3

,sum(GOAL) filter(where Ev_Zone='Off') as "2nd_Assists"

FROM Player_Events
GROUP BY p3_name, p3_ID, Season_id
)

,merged_data as (
SELECT


p1_stats.*
,p2_stats.*
,p3_stats.*



FROM p1_stats LEFT JOIN p2_stats
ON p1_stats.Player_Id=p2_stats.Player_Id2 and p1_stats.Season_id=p2_stats.Season_id2
LEFT JOIN p3_stats
ON p2_stats.Player_Id2=p3_stats.Player_Id3 and p2_stats.Season_id2=p3_stats.Season_id3
)

SELECT


merged_data.*


,'Goals'+'Shots_on_Net (no goal)'+'Shots_for_miss'+'Shot_Attempts_Leading_to_Block' as "Total_Shot_Attempts"
,'Goals'/('Goals'+'Shots_on_Net (no goal)'+'Shots_for_miss'+'Shot_Attempts_Leading_to_Block') as "True_Shooting_Pct"
,('Goals'+'Shots_on_Net (no goal)')/('Goals'+'Shots_on_Net (no goal)'+'Shots_for_miss'+'Shot_Attempts_Leading_to_Block') as "Shot_on_Net_Pct"
,"Goals"-"xG" as "Actual_to_Expected_Goals"
,"Goals"-"xG"/"xG" as "Actual_to_Expected_Goals_%"
,'Shot_Distance_Total'/('Goals'+'Shots_on_Net (no goal)'+'Shots_for_miss'+'Shot_Attempts_Leading_to_Block') as "Average_Shot_Distance"

,DZ_Giveaways+NZ_Giveaways+OZ_Giveaways as "Total_Giveaways"
,DZ_Hits+NZ_Hits+OZ_Hits as "Total_Hits"
,DZ_Penalties_Taken+NZ_Penalties_Taken+OZ_Penalties_Taken as "Total_Penalties_Taken"
,DZ_Takeaways+NZ_Takeaways+OZ_Takeaways as "Total_Takeaways"

,DZ_Faceoff_Won/(DZ_Faceoff_Won+DZ_Faceoff_Lost) as "DZ_Faceoff_Win_Pct"
,NZ_Faceoff_Won/(NZ_Faceoff_Won+NZ_Faceoff_Lost) as "NZ_Faceoff_Win_Pct"
,OZ_Faceoff_Won/(OZ_Faceoff_Won+OZ_Faceoff_Lost) as "OZ_Faceoff_Win_Pct"
,(DZ_Faceoff_Won+NZ_Faceoff_Won+OZ_Faceoff_Won)/(DZ_Faceoff_Won+NZ_Faceoff_Won+OZ_Faceoff_Won+DZ_Faceoff_Lost+NZ_Faceoff_Lost+OZ_Faceoff_Lost) as "Faceoff_Win_Pct"

from merged_data

 """, NHL_DATA_conn)

In [78]:
NHL_PLAYER_STATS = NHL_PLAYER_STATS[['Player', 'Player_Id', 'Season_id', 'Games_Played', 'DZ_Blocks',
       'DZ_Faceoff_Won', 'DZ_Giveaways', 'DZ_Hits', 'DZ_Penalties_Taken',
       'DZ_Takeaways', 'NZ_Faceoff_Won', 'NZ_Giveaways', 'NZ_Hits',
       'NZ_Penalties_Taken', 'NZ_Takeaways', 'Rebound_Shot_Attempts',
       'Shot_Attempts_OFf_Rush', 'Shot_Attempts_Off_FaceOff',
       'Shot_Attempts_Off_Takeaway', 'DEFLECTED_Shot_Attempts',
       'Royal_Road_Shot_Attempts', 'Shot_Distance_Total', 'OZ_Faceoff_Won',
       'OZ_Giveaways', 'OZ_Hits', 'OZ_Penalties_Taken', 'OZ_Takeaways',
       'Goals', 'xG', 'Shots_on_Net (no goal)', 'Shots_for_miss', 'DZ_Faceoff_Lost', 'DZ_Hits_Recieved',
       'DZ_Penalties_Drawn', 'NZ_Faceoff_Lost', 'NZ_Hits_Recieved',
       'NZ_Penalties_Drawn', 'OZ_Faceoff_Lost', 'OZ_Hits_Recieved',
       'OZ_Penalties_Drawn', '1st_Assists', 'Shot_Attempts_Leading_to_Block', '2nd_Assists',
       'Total_Shot_Attempts', 'True_Shooting_Pct', 'Shot_on_Net_Pct',
       'Actual_to_Expected_Goals', 'Actual_to_Expected_Goals_%',
       'Average_Shot_Distance', 'Total_Giveaways', 'Total_Hits',
       'Total_Penalties_Taken', 'Total_Takeaways', 'DZ_Faceoff_Win_Pct',
       'NZ_Faceoff_Win_Pct', 'OZ_Faceoff_Win_Pct', 'Faceoff_Win_Pct']]

NHL_PLAYER_STATS

,Player,Player_Id,Season_id,Games_Played,DZ_Blocks,DZ_Faceoff_Won,DZ_Giveaways,DZ_Hits,DZ_Penalties_Taken,DZ_Takeaways,...,Actual_to_Expected_Goals_%,Average_Shot_Distance,Total_Giveaways,Total_Hits,Total_Penalties_Taken,Total_Takeaways,DZ_Faceoff_Win_Pct,NZ_Faceoff_Win_Pct,OZ_Faceoff_Win_Pct,Faceoff_Win_Pct
0,A.J. GREER,8478421.0,2016,5,3.0,0.0,0.0,0.0,0.0,1.0,...,-1.0,None,2.0,4.0,2.0,1.0,NaN,NaN,NaN,NaN
1,A.J. GREER,8478421.0,2017,16,5.0,1.0,0.0,10.0,3.0,2.0,...,-1.0,None,2.0,30.0,9.0,3.0,0.0,NaN,0.0,0.0
2,A.J. GREER,8478421.0,2018,14,5.0,1.0,0.0,3.0,2.0,2.0,...,0.0,None,1.0,20.0,7.0,4.0,0.0,1.0,NaN,0.0
3,A.J. GREER,8478421.0,2020,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A.J. GREER,8478421.0,2021,9,3.0,0.0,3.0,3.0,1.0,0.0,...,0.0,None,4.0,20.0,1.0,0.0,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9963,ZEMGUS GIRGENSONS,8476878.0,2018,72,27.0,17.0,4.0,39.0,4.0,6.0,...,4.0,None,11.0,144.0,7.0,21.0,0.0,0.0,0.0,0.0
9964,ZEMGUS GIRGENSONS,8476878.0,2019,69,20.0,15.0,6.0,23.0,3.0,5.0,...,11.0,None,11.0,110.0,5.0,13.0,0.0,0.0,0.0,0.0
9965,ZEMGUS GIRGENSONS,8476878.0,2021,55,24.0,72.0,2.0,28.0,3.0,8.0,...,9.0,None,10.0,81.0,8.0,24.0,0.0,0.0,0.0,0.0
9966,ZEMGUS GIRGENSONS,8476878.0,2022,72,43.0,45.0,3.0,28.0,2.0,9.0,...,8.0,None,7.0,93.0,7.0,19.0,0.0,0.0,0.0,0.0


In [79]:
NHL_PLAYER_STATS = NHL_PLAYER_STATS.fillna(0)
NHL_PLAYER_STATS['Player_Id'] = NHL_PLAYER_STATS['Player_Id'].astype(int)
NHL_PLAYER_STATS

,Player,Player_Id,Season_id,Games_Played,DZ_Blocks,DZ_Faceoff_Won,DZ_Giveaways,DZ_Hits,DZ_Penalties_Taken,DZ_Takeaways,...,Actual_to_Expected_Goals_%,Average_Shot_Distance,Total_Giveaways,Total_Hits,Total_Penalties_Taken,Total_Takeaways,DZ_Faceoff_Win_Pct,NZ_Faceoff_Win_Pct,OZ_Faceoff_Win_Pct,Faceoff_Win_Pct
0,A.J. GREER,8478421,2016,5,3.0,0.0,0.0,0.0,0.0,1.0,...,-1.0,0,2.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0
1,A.J. GREER,8478421,2017,16,5.0,1.0,0.0,10.0,3.0,2.0,...,-1.0,0,2.0,30.0,9.0,3.0,0.0,0.0,0.0,0.0
2,A.J. GREER,8478421,2018,14,5.0,1.0,0.0,3.0,2.0,2.0,...,0.0,0,1.0,20.0,7.0,4.0,0.0,1.0,0.0,0.0
3,A.J. GREER,8478421,2020,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A.J. GREER,8478421,2021,9,3.0,0.0,3.0,3.0,1.0,0.0,...,0.0,0,4.0,20.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9963,ZEMGUS GIRGENSONS,8476878,2018,72,27.0,17.0,4.0,39.0,4.0,6.0,...,4.0,0,11.0,144.0,7.0,21.0,0.0,0.0,0.0,0.0
9964,ZEMGUS GIRGENSONS,8476878,2019,69,20.0,15.0,6.0,23.0,3.0,5.0,...,11.0,0,11.0,110.0,5.0,13.0,0.0,0.0,0.0,0.0
9965,ZEMGUS GIRGENSONS,8476878,2021,55,24.0,72.0,2.0,28.0,3.0,8.0,...,9.0,0,10.0,81.0,8.0,24.0,0.0,0.0,0.0,0.0
9966,ZEMGUS GIRGENSONS,8476878,2022,72,43.0,45.0,3.0,28.0,2.0,9.0,...,8.0,0,7.0,93.0,7.0,19.0,0.0,0.0,0.0,0.0


In [80]:
NHL_PLAYER_STATS.to_csv('NHL_Player_Metrics_PBP.csv', index=False)

In [17]:
NHL_Dataset = pd.read_sql_query(
"""
with add_bio as (
SELECT 
Player_Metrics.*

,Player_Bio.Birth_Year as Birth_Year
,Player_Bio.Birth_Month as Birth_Month

,Player_Bio.Position as Position
,Player_Bio.Handedness as Handedness

,Player_Bio.Draft_Year as Draft_Year
,Player_Bio.Draft_Team as Draft_Team
,Player_Bio.Draft_Position as "Draft_Rank"


,(Player_Metrics.Season_id)-(Player_Bio.Birth_Year) as Age

FROM Player_Metrics LEFT JOIN Player_Bio
ON Player_Metrics.Player_Id=Player_Bio.Player_Id
)

,final_merge as (
SELECT

add_bio.*

,Player_Shifts.TOI_seconds as TOI_seconds
,Player_Shifts.Game_Count as Game_Count

,Player_RAPM.Offensive_RAPM
,Player_RAPM.Defensive_RAPM
,Player_RAPM.total


FROM add_bio 
LEFT JOIN Player_Shifts
ON add_bio.Player_Id=Player_Shifts.Player_Id and add_bio.Season_id=Player_Shifts.Season_id
LEFT JOIN Player_RAPM
ON Player_Shifts.Player_Id=Player_RAPM.player_id and Player_Shifts.Season_id=Player_RAPM.Season
)

SELECT

final_merge.*
,(Offensive_RAPM/3600)*TOI_seconds as "Offensive_RAPM_Total"
,(Defensive_RAPM/3600)*TOI_seconds as "Defensive_RAPM_Total"
,(total/3600)*TOI_seconds as "Overall_RAPM_Total"

FROM final_merge
WHERE  Player_Id>0
ORDER BY Player_ID, Season_id DESC;
 """, NHL_DATA_conn)
NHL_Dataset

,index,Player,Player_Id,Season_id,Games_Played,DZ_Blocks,DZ_Faceoff_Won,DZ_Giveaways,DZ_Hits,DZ_Penalties_Taken,...,Draft_Rank,Age,TOI_seconds,Game_Count,Offensive_RAPM,Defensive_RAPM,total,Offensive_RAPM_Total,Defensive_RAPM_Total,Overall_RAPM_Total
0,5355,JAROMIR JAGR,8448208,2017,19,8,0,3,0,2,...,5,45.0,16549.0,21,-0.005143,-0.079581,0.074438,-0.023640,-0.365829,0.342189
1,855,JAROMIR JAGR,8448208,2016,80,4,0,4,8,10,...,5,44.0,83639.0,82,0.163225,0.233260,-0.070035,3.792223,5.419345,-1.627122
2,769,JAROMIR JAGR,8448208,2015,82,19,0,12,12,10,...,5,43.0,88799.0,85,0.184476,-0.221254,0.405730,4.550355,-5.457528,10.007883
3,970,JAROMIR JAGR,8448208,2014,76,14,1,14,5,9,...,5,42.0,80191.0,76,0.196924,-0.175597,0.372520,4.386526,-3.911463,8.297989
4,277,JAROMIR JAGR,8448208,2013,82,8,0,5,1,8,...,5,41.0,94259.0,82,0.488825,-0.192685,0.681510,12.798925,-5.045078,17.844003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9969,6913,MARCUS BJORK,8483620,2022,29,43,0,8,36,10,...,undrafted,25.0,32547.0,29,0.009806,0.316242,-0.306435,0.088659,2.859091,-2.770432
9970,5842,PAVOL REGENDA,8483630,2022,13,13,0,1,4,0,...,undrafted,23.0,8855.0,14,0.124723,-0.137464,0.262186,0.306783,-0.338123,0.644906
9971,8846,ANTON LEVTCHI,8483641,2022,1,0,0,1,0,0,...,undrafted,27.0,1149.0,2,-0.114701,0.132438,-0.247140,-0.036609,0.042270,-0.078879
9972,858,ANDREI KUZMENKO,8483808,2022,69,18,0,6,4,3,...,undrafted,26.0,71971.0,74,0.166479,-0.209239,0.375718,3.328238,-4.183093,7.511331


In [24]:
NHL_Dataset = NHL_Dataset.fillna(0)
NHL_Dataset

,index,Player,Player_Id,Season_id,Games_Played,DZ_Blocks,DZ_Faceoff_Won,DZ_Giveaways,DZ_Hits,DZ_Penalties_Taken,...,Draft_Rank,Age,TOI_seconds,Game_Count,Offensive_RAPM,Defensive_RAPM,total,Offensive_RAPM_Total,Defensive_RAPM_Total,Overall_RAPM_Total
0,0,CONNOR MCDAVID,8478402,2021,96,29,209,22,35,12,...,1,24.0,129420.0,96.0,1.129378,-0.119175,1.248553,40.601146,-4.284330,44.885476
1,1,ALEX OVECHKIN,8471214,2017,106,35,1,28,48,9,...,1,32.0,128956.0,106.0,0.428871,0.058674,0.370197,15.362641,2.101760,13.260881
2,2,ALEX OVECHKIN,8471214,2015,91,38,0,13,65,10,...,1,30.0,111615.0,91.0,0.437336,-0.277870,0.715206,13.559248,-8.615133,22.174381
3,3,ALEX OVECHKIN,8471214,2014,95,39,0,26,84,11,...,1,29.0,115544.0,95.0,0.301943,0.045019,0.256924,9.691044,1.444923,8.246121
4,4,ZACH HYMAN,8475786,2022,74,16,1,4,13,7,...,123,30.0,90269.0,74.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9990,9963,YANN DANIS,8471186,2015,1,0,0,1,0,0,...,0,0.0,3030.0,2.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9991,9964,YAROSLAV ASKAROV,8482137,2022,1,0,0,1,0,0,...,0,0.0,3470.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9992,9965,ZAC DALPE,8474610,2021,1,1,0,0,0,0,...,45,32.0,368.0,1.0,-0.059759,-0.032023,-0.027736,-0.006109,-0.003273,-0.002835
9993,9966,ZACH SAWCHENKO,8479313,2021,1,0,0,1,0,0,...,0,0.0,17352.0,7.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [25]:
NHL_Dataset.to_csv('NHL_Player_Metrics_COUNTS.csv', index=False)

In [18]:
NHL_Dataset_p20 = NHL_Dataset

In [19]:
NHL_Dataset_p20.columns

Index(['index', 'Player', 'Player_Id', 'Season_id', 'Games_Played',
       'DZ_Blocks', 'DZ_Faceoff_Won', 'DZ_Giveaways', 'DZ_Hits',
       'DZ_Penalties_Taken', 'DZ_Takeaways', 'NZ_Faceoff_Won', 'NZ_Giveaways',
       'NZ_Hits', 'NZ_Penalties_Taken', 'NZ_Takeaways',
       'Rebound_Shot_Attempts', 'Shot_Attempts_OFf_Rush',
       'Shot_Attempts_Off_FaceOff', 'Shot_Attempts_Off_Takeaway',
       'DEFLECTED_Shot_Attempts', 'Royal_Road_Shot_Attempts',
       'Shot_Distance_Total', 'OZ_Faceoff_Won', 'OZ_Giveaways', 'OZ_Hits',
       'OZ_Penalties_Taken', 'OZ_Takeaways', 'Goals', 'xG',
       'Shots_on_Net (no goal)', 'Shots_for_miss', 'DZ_Faceoff_Lost',
       'DZ_Hits_Recieved', 'DZ_Penalties_Drawn', 'NZ_Faceoff_Lost',
       'NZ_Hits_Recieved', 'NZ_Penalties_Drawn', 'OZ_Faceoff_Lost',
       'OZ_Hits_Recieved', 'OZ_Penalties_Drawn', '1st_Assists',
       'Shot_Attempts_Leading_to_Block', '2nd_Assists', 'Total_Shot_Attempts',
       'True_Shooting_Pct', 'Shot_on_Net_Pct', 'Actual_to_Expe

In [20]:
col_list = ['DZ_Blocks', 'DZ_Faceoff_Won', 'DZ_Giveaways', 'DZ_Hits',
       'DZ_Penalties_Taken', 'DZ_Takeaways', 'NZ_Faceoff_Won', 'NZ_Giveaways',
       'NZ_Hits', 'NZ_Penalties_Taken', 'NZ_Takeaways',
       'Rebound_Shot_Attempts', 'Shot_Attempts_OFf_Rush',
       'Shot_Attempts_Off_FaceOff', 'Shot_Attempts_Off_Takeaway',
       'DEFLECTED_Shot_Attempts', 'Royal_Road_Shot_Attempts',
       'Shot_Distance_Total', 'OZ_Faceoff_Won', 'OZ_Giveaways', 'OZ_Hits',
       'OZ_Penalties_Taken', 'OZ_Takeaways', 'Goals', 'xG',
       'Shots_on_Net (no goal)', 'Shots_for_miss', 'DZ_Faceoff_Lost',
       'DZ_Hits_Recieved', 'DZ_Penalties_Drawn', 'NZ_Faceoff_Lost',
       'NZ_Hits_Recieved', 'NZ_Penalties_Drawn', 'OZ_Faceoff_Lost',
       'OZ_Hits_Recieved', 'OZ_Penalties_Drawn', '1st_Assists',
       'Shot_Attempts_Leading_to_Block', '2nd_Assists', 'Total_Shot_Attempts','Actual_to_Expected_Goals',
       'Total_Giveaways', 'Total_Hits', 'Total_Penalties_Taken',
       'Total_Takeaways']

In [21]:
for i in col_list:
    NHL_Dataset_p20[f'{i}_p20'] = (NHL_Dataset_p20[f'{i}']/NHL_Dataset_p20['TOI_seconds'])*1200
    NHL_Dataset_p20 = NHL_Dataset_p20.drop(f'{i}' ,axis=1)

NHL_Dataset_p20

,index,Player,Player_Id,Season_id,Games_Played,True_Shooting_Pct,Shot_on_Net_Pct,Actual_to_Expected_Goals_%,Average_Shot_Distance,DZ_Faceoff_Win_Pct,...,OZ_Penalties_Drawn_p20,1st_Assists_p20,Shot_Attempts_Leading_to_Block_p20,2nd_Assists_p20,Total_Shot_Attempts_p20,Actual_to_Expected_Goals_p20,Total_Giveaways_p20,Total_Hits_p20,Total_Penalties_Taken_p20,Total_Takeaways_p20
0,5355,JAROMIR JAGR,8448208,2017,19,0.019607843,0.607843137,0,16.79416635,0.000000,...,0.000000,0.290048,0.870143,0.145024,3.698109,-0.171598,0.870143,0.072512,0.290048,0.797631
1,855,JAROMIR JAGR,8448208,2016,80,0.051724138,0.613793103,14,19.49203637,0.000000,...,0.043042,0.286947,0.746063,0.143474,4.160738,-0.078656,0.631284,0.301295,0.401726,0.573895
2,769,JAROMIR JAGR,8448208,2015,82,0.089347079,0.563573883,25,19.99423954,0.000000,...,0.013514,0.297301,0.716224,0.256760,3.932477,0.063463,0.635142,0.459465,0.337842,0.513519
3,970,JAROMIR JAGR,8448208,2014,76,0.050167224,0.56187291,14,19.14041268,1.000000,...,0.000000,0.299285,1.107356,0.134678,4.474318,-0.067685,0.688357,0.433964,0.269357,0.568642
4,277,JAROMIR JAGR,8448208,2013,82,0.063938619,0.601023018,24,19.33654178,0.000000,...,0.025462,0.280079,0.967547,0.254618,4.977774,-0.033104,0.751122,0.050924,0.292810,0.458312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9969,6913,MARCUS BJORK,8483620,2022,29,0.052631579,0.526315789,2,35.6931829,0.000000,...,0.073740,0.184349,0.589916,0.110609,2.101576,0.066753,0.368698,1.437921,0.442437,0.073740
9970,5842,PAVOL REGENDA,8483630,2022,13,0.034482759,0.586206897,0,22.00709682,0.000000,...,0.135517,0.135517,0.406550,0.135517,3.929983,-0.213020,0.677583,1.084133,0.271033,0.542067
9971,8846,ANTON LEVTCHI,8483641,2022,1,0,1,-1,41.86884283,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.044386,-0.023817,2.088773,0.000000,0.000000,0.000000
9972,858,ANDREI KUZMENKO,8483808,2022,69,0.15625,0.53515625,39,15.09408155,0.000000,...,0.033347,0.216754,0.967056,0.316794,4.268386,0.325735,0.400161,0.216754,0.066694,0.500201


In [22]:
NHL_Dataset_p20.to_csv('NHL_Player_Metrics_P20.csv', index=False)